In [1]:
import lseg.data as ld  # Importa la librería de datos LSEG (Refinitiv) para descargar históricos
import numpy as np       # Importa NumPy para cálculos numéricos y simulación
import pandas as pd      # Importa pandas para manipulación de datos
import matplotlib.pyplot as plt  # Importa Matplotlib para visualizaciones

# --- Configuración ---
ld.open_session()  # Abre la sesión con LSEG para poder realizar consultas de datos

<lseg.data.session.Definition object at 0x139df6cf7d0 {name='workspace'}>

In [ ]:
# Entradas del análisis
import numpy as np
import pandas as pd
from datetime import datetime

# Tickers (RICs de México en Refinitiv)
tickers = ["HOTEL.MX", "PINFRA.MX", "TLEVISACPO.MX", "GFINBURO.MX", "GCARSOA1.MX"]
# Pesos provistos (se normalizan por seguridad)
pesos = np.array([0.1, 0.2, 0.15, 0.3, 0.25], dtype=float)
pesos = pesos / pesos.sum()

# Rango de fechas solicitado
start_date = "2025-09-08"
end_date = "2025-09-26"

print("Tickers:", tickers)
print("Pesos:", pesos)
print(f"Periodo: {start_date} a {end_date}")

In [ ]:
# Descarga de precios desde Refinitiv (LSEG)
from typing import List

# Se asume que la sesión ya está abierta en la primer celda con ld.open_session()
# Campo de cierre a usar (consistente con tu repo)
FIELD_CLOSE = "TR.PriceClose"

try:
    # get_history puede aceptar lista de RICs y devolver DataFrame con MultiIndex
    raw = ld.get_history(
        universe=tickers,
        fields=[FIELD_CLOSE],
        interval="1D",
        start=start_date,
        end=end_date,
    )
    # Normalizar a formato ancho: columnas por ticker con el precio de cierre
    if isinstance(raw.columns, pd.MultiIndex):
        # se espera algo tipo ('Price Close', 'HOTEL.MX'); armamos un pivot limpio
        if ("Price Close" in raw.columns.get_level_values(0)):
            close = raw["Price Close"].copy()
        else:
            # fallback si el título es diferente (p.ej., TR.PriceClose)
            lvl0 = raw.columns.get_level_values(0)
            maybe = [c for c in set(lvl0) if "Price" in str(c) or "Close" in str(c)]
            key = maybe[0] if maybe else lvl0[0]
            close = raw[key].copy()
    else:
        # Columnas simples, renombrar a 'Price Close' y luego a ticker si corresponde
        # Cuando universe es 1, suele regresar una sola columna Price Close
        close = raw.rename(columns={"Price Close": tickers[0]})

    # Aseguramos que el índice sea de fecha y ordenado
    close.index = pd.to_datetime(close.index)
    close = close.sort_index()

    # Filtramos exactamente el rango (por seguridad)
    mask = (close.index >= pd.to_datetime(start_date)) & (close.index <= pd.to_datetime(end_date))
    prices = close.loc[mask]

    # Si hay huecos de días sin cotización, forward-fill para calcular rendimientos diarios válidos
    prices = prices.asfreq("B")  # frecuencia días hábiles
    prices = prices.ffill()

    display(prices.tail())
except Exception as e:
    print("Error descargando datos de LSEG:", e)
    prices = None

In [ ]:
# Cálculo de rendimientos y métricas de riesgo

if prices is None or prices.empty:
    raise ValueError("No hay precios descargados; revisa la celda anterior.")

# Rendimientos diarios por acción (log-returns para estabilidad numérica)
retn = np.log(prices / prices.shift(1)).dropna(how="all")

# Estadísticos por acción en el periodo: retorno acumulado, media diaria, varianza y desviación
stats_accion = pd.DataFrame(index=retn.columns)
stats_accion["ret_acum_%"] = (prices.iloc[-1] / prices.iloc[0] - 1.0) * 100
stats_accion["ret_diario_prom_%"] = (retn.mean() * 100)
stats_accion["var_diaria"] = retn.var(ddof=1)
stats_accion["std_diaria"] = retn.std(ddof=1)

# Métricas de portafolio
w = pd.Series(pesos, index=retn.columns)
# Ajuste: si falta alguna columna por no cotizar, igualamos pesos a columnas disponibles
w = w.reindex(retn.columns).fillna(0)
# Normalizar si la suma cambia por columnas faltantes
if w.sum() != 0:
    w = w / w.sum()

# Retornos diarios del portafolio (lineales para agregar) usando log-returns aproximados
ret_port = (retn @ w)
ret_port_acum = np.exp(ret_port.cumsum()).iloc[-1] - 1.0
var_port = ret_port.var(ddof=1)
std_port = ret_port.std(ddof=1)

# Matriz de covarianza (para referencia)
cov = retn.cov()

print("Resumen por acción (periodo 2025-09-08 a 2025-09-26):")
display(stats_accion.round({"ret_acum_%": 3, "ret_diario_prom_%": 4, "var_diaria": 8, "std_diaria": 4}))

print("\nResumen del portafolio:")
print({
    "ret_acum_%": round(ret_port_acum * 100, 3),
    "ret_diario_prom_%": round(ret_port.mean() * 100, 4),
    "var_diaria": round(var_port, 8),
    "std_diaria": round(std_port, 4),
})

In [ ]:
# Tablas y gráficos rápidos
import matplotlib.pyplot as plt

# Tabla de covarianza y correlación (opcional)
display(cov.round(8))
display(retn.corr().round(4))

# Grafiquemos la trayectoria del portafolio en el periodo
(1 + ret_port).cumprod().plot(title="Evolución del Portafolio (base=1)", figsize=(8,3))
plt.ylabel("Índice (base=1)")
plt.show()

# Y los precios para cada acción
prices.plot(title="Precios de cierre (MX)", figsize=(9,4))
plt.ylabel("Precio")
plt.show()

In [7]:
import lseg.data as ld  # Refinitiv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# --- Configuración ---
ld.open_session()  # abrir sesión de Refinitiv

tickers = ["HOTEL.MX", "PINFRA.MX", "TLEVISACPO.MX", "GFINBURO.MX", "GCARSOA1.MX"]
pesos = np.array([0.1, 0.2, 0.15, 0.3, 0.25])

# --- Descargar precios históricos ---
precios = ld.get_history(
    universe=tickers,
    fields="TR.PriceClose",
    start="2025-09-08",
    end="2025-09-26",
    interval="1d"
)

# Ajustar dataframe (manejo robusto para distintos formatos que devuelve ld.get_history)
# Puede venir como MultiIndex con nivel 'TR.PriceClose' o 'Price Close', o ya en formato ancho por ticker.
if isinstance(precios.columns, pd.MultiIndex):
    lvl0 = precios.columns.get_level_values(0)
    if "TR.PriceClose" in lvl0:
        precios = precios["TR.PriceClose"].copy()
    elif "Price Close" in lvl0:
        precios = precios["Price Close"].copy()
    else:
        # buscar algún nivel que contenga 'Price' o 'Close'
        maybe = [c for c in set(lvl0) if "Price" in str(c) or "Close" in str(c)]
        key = maybe[0] if maybe else lvl0[0]
        precios = precios[key].copy()

    # Si aún quedan columnas multi-nivel, intentar unstack para tener columnas por ticker
    if isinstance(precios.columns, pd.MultiIndex):
        try:
            precios = precios.unstack(level=0)
        except Exception:
            try:
                precios = precios.swaplevel(0, 1, axis=1).unstack(level=0)
            except Exception:
                # dejar como está y avisar
                print("Advertencia: no se pudo unstackear completamente 'precios'; verifique la estructura.")
else:
    # columnas simples: si existe una columna llamada 'TR.PriceClose' o 'Price Close', convertir a DataFrame
    if "TR.PriceClose" in precios.columns:
        precios = precios["TR.PriceClose"].to_frame()
    elif "Price Close" in precios.columns:
        precios = precios["Price Close"].to_frame()
    # si ya está en formato ancho por tickers, no hacer nada

# Normalizar nombres de columnas (quitar nivel extra si existe) y asegurar strings
if isinstance(precios.columns, pd.MultiIndex):
    precios.columns = [c[1] if len(c) > 1 else c[0] for c in precios.columns]
precios.columns = [str(c) for c in precios.columns]

# Reordenar/filtrar columnas según la lista original de tickers si están disponibles
if set(tickers).issubset(set(precios.columns)):
    precios = precios[tickers]

# --- Calcular rendimientos diarios ---
rendimientos = precios.pct_change().dropna()

# --- Estadísticas individuales ---
rend_ind = rendimientos.mean()
var_ind = rendimientos.var()
std_ind = rendimientos.std()

resumen_ind = pd.DataFrame({
    "Rendimiento promedio": rend_ind,
    "Varianza": var_ind,
    "Desv. Estándar": std_ind
})

print("Resultados por acción:")
print(resumen_ind)

# --- Portafolio ---
# Rendimiento esperado del portafolio
rend_portafolio = np.dot(pesos, rendimientos.mean())

# Invertir US$1,000,000 en el portafolio (opción)
capital_usd = 1_000_000.0
# Si los precios están en otra divisa (p.ej. MXN), ajustar tipo de cambio: MXN por 1 USD
fx_rate = 18.5  # cambiar a p.ej. 18.5 si los precios están en MXN

capital_local = capital_usd * fx_rate

# Serie de pesos alineada con las columnas disponibles
w_ser = pd.Series(pesos, index=precios.columns).reindex(precios.columns).fillna(0.0)
if w_ser.sum() != 0:
    w_ser = w_ser / w_ser.sum()

alloc_local = w_ser * capital_local
last_price = precios.iloc[-1].astype(float)

# Compra con número entero de acciones
shares_int = np.floor(alloc_local / last_price).astype(int)
invested_local = shares_int * last_price
cash_remaining = capital_local - invested_local.sum()

# Opción con fracciones (si el broker lo permite)
shares_frac = alloc_local / last_price
invested_frac = shares_frac * last_price  # coincide con alloc_local

df_alloc = pd.DataFrame({
    "Peso objetivo": w_ser,
    "Asignación (local)": alloc_local,
    "Último precio": last_price,
    "Acciones (enteras)": shares_int,
    "Invertido (enteras)": invested_local,
    "Acciones (fracc.)": shares_frac,
    "Invertido (fracc.)": invested_frac
})

print(f"\nSimulación de inversión: US${capital_usd:,.0f} (fx={fx_rate})")
display(df_alloc)
print(f"\nTotal invertido (enteras): {invested_local.sum():,.2f}")
print(f"Efectivo sobrante: {cash_remaining:,.2f}")

# Recalcular métricas del portafolio usando las ponderaciones reales por monto invertido
if invested_local.sum() > 0:
    pos_weights = invested_local / invested_local.sum()
else:
    pos_weights = pd.Series(0.0, index=precios.columns)

port_return_est = (rendimientos.mean() * pos_weights).sum()
port_var_est = np.dot(pos_weights.T, np.dot(cov_matrix, pos_weights))
port_std_est = np.sqrt(port_var_est)

print("\nEstimado con ponderaciones reales (por monto invertido):")
print(f"Rendimiento esperado diario (aprox): {port_return_est:.6f}")
print(f"Varianza (diaria): {port_var_est:.6f}")
print(f"Desv. Estándar (diaria): {port_std_est:.6f}")
cov_matrix = rendimientos.cov()
var_portafolio = np.dot(pesos.T, np.dot(cov_matrix, pesos))

# Desviación estándar del portafolio
std_portafolio = np.sqrt(var_portafolio)

print("\nResultados del portafolio:")

# Rendimientos diarios (ya calculados): promedio por acción
print("\nRendimiento promedio diario (usado en el cálculo):")
print(rend_ind.round(6))

# Rendimiento total del periodo: (último - primero) / primero
rend_total_periodo = (precios.iloc[-1] - precios.iloc[0]) / precios.iloc[0]
print("\nRendimiento total del periodo (último vs primer precio):")
print(rend_total_periodo.round(6))

# Rendimiento total del portafolio según pesos objetivo (fraccionales)
port_total_by_weights = (rend_total_periodo * w_ser).sum()
print(f"\nRendimiento total del portafolio (por pesos objetivo): {port_total_by_weights:.6f}")

# Rendimiento total "real" según número entero de acciones compradas
base_inicial = (precios.iloc[0] * shares_int).sum()
base_final = (precios.iloc[-1] * shares_int).sum()
if base_inicial > 0:
    port_total_by_shares_int = (base_final - base_inicial) / base_inicial
else:
    port_total_by_shares_int = np.nan
print(f"Rendimiento total del portafolio (según acciones enteras): {port_total_by_shares_int:.6f}")
print(f"Rendimiento esperado: {rend_portafolio:.6f}")
print(f"Varianza: {var_portafolio:.6f}")
print(f"Desv. Estándar: {std_portafolio:.6f}")

Resultados por acción:
               Rendimiento promedio  Varianza  Desv. Estándar
HOTEL.MX                   0.001569   0.00002         0.00451
PINFRA.MX                   0.00059  0.000228        0.015102
TLEVISACPO.MX             -0.004957  0.000346        0.018613
GFINBURO.MX               -0.000011  0.000205        0.014314
GCARSOA1.MX                0.002584  0.000552        0.023503

Simulación de inversión: US$1,000,000 (fx=18.5)


,Peso objetivo,Asignación (local),Último precio,Acciones (enteras),Invertido (enteras),Acciones (fracc.),Invertido (fracc.)
HOTEL.MX,0.10,1850000.0,3.49,530085,1849996.65,530085.959885,1850000.0
PINFRA.MX,0.20,3700000.0,244.93,15106,3699912.58,15106.356918,3700000.0
TLEVISACPO.MX,0.15,2775000.0,9.86,281440,2774998.40,281440.162272,2775000.0
GFINBURO.MX,0.30,5550000.0,51.04,108738,5549987.52,108738.244514,5550000.0
GCARSOA1.MX,0.25,4625000.0,132.66,34863,4624925.58,34863.560983,4625000.0



Total invertido (enteras): 18,499,820.73
Efectivo sobrante: 179.27

Estimado con ponderaciones reales (por monto invertido):
Rendimiento esperado diario (aprox): 0.000174
Varianza (diaria): 0.000110
Desv. Estándar (diaria): 0.010485

Resultados del portafolio:

Rendimiento promedio diario (usado en el cálculo):
HOTEL.MX         0.001569
PINFRA.MX         0.00059
TLEVISACPO.MX   -0.004957
GFINBURO.MX     -0.000011
GCARSOA1.MX      0.002584
dtype: Float64

Rendimiento total del periodo (último vs primer precio):
HOTEL.MX         0.020468
PINFRA.MX        0.006327
TLEVISACPO.MX   -0.064516
GFINBURO.MX      -0.00137
GCARSOA1.MX      0.030689
dtype: Float64

Rendimiento total del portafolio (por pesos objetivo): 0.000896
Rendimiento total del portafolio (según acciones enteras): -0.000049
Rendimiento esperado: 0.000174
Varianza: 0.000110
Desv. Estándar: 0.010485


In [8]:
import pandas as pd
import numpy as np

# Creando un DataFrame con los datos que proporcionaste
data = {
    'Ticker': ["GCARSOA1", "GFINBUR", "TELEVISA", "PINFRA", "HOTEL"],
    'Precio Inicial': [126.38, 51.05, 10.37, 243.89, 3.45],
    'Precio Final': [131.26, 51.83, 10.09, 246.08, 3.5],
    'Peso': [0.25, 0.3, 0.15, 0.2, 0.1]
}
df = pd.DataFrame(data)

# 1. Calcular el rendimiento para cada acción
df['Rendimiento'] = (df['Precio Final'] - df['Precio Inicial']) / df['Precio Inicial']

# 2. Calcular el rendimiento ponderado para cada acción
df['Rendimiento Ponderado'] = df['Rendimiento'] * df['Peso']

# 3. Calcular el rendimiento total del portafolio sumando los ponderados
rendimiento_portafolio = df['Rendimiento Ponderado'].sum()

# --- Impresión de Resultados ---
print("Rendimiento por cada acción (basado en tus datos):\n")
for index, row in df.iterrows():
    print(f"{row['Ticker']}: {row['Rendimiento']:.2%}")

print("\n" + "="*40)
print(f"\nRendimiento total del portafolio: {rendimiento_portafolio:.2%}")

Rendimiento por cada acción (basado en tus datos):

GCARSOA1: 3.86%
GFINBUR: 1.53%
TELEVISA: -2.70%
PINFRA: 0.90%
HOTEL: 1.45%


Rendimiento total del portafolio: 1.34%


In [10]:
import pandas as pd
import numpy as np

# Creando un DataFrame con los datos que proporcionaste
data = {
    'Ticker': ["GCARSOA1", "GFINBUR", "TELEVISA", "PINFRA", "HOTEL"],
    'Precio Inicial': [126.38, 51.05, 10.37, 243.89, 3.45],
    'Precio Final': [131.26, 51.83, 10.09, 246.08, 3.5],
    'Peso': [0.25, 0.3, 0.15, 0.2, 0.1]
}
df = pd.DataFrame(data)

# 1. Calcular el rendimiento para cada acción
df['Rendimiento'] = (df['Precio Final'] - df['Precio Inicial']) / df['Precio Inicial']

# 2. Calcular el rendimiento ponderado para cada acción
df['Rendimiento Ponderado'] = df['Rendimiento'] * df['Peso']

# 3. Calcular el rendimiento total del portafolio sumando los ponderados
rendimiento_portafolio = df['Rendimiento Ponderado'].sum()

# 4. Calcular varianza y desviación estándar de los rendimientos individuales
varianza = df['Rendimiento'].var()
desviacion = df['Rendimiento'].std()

# --- Impresión de Resultados ---
print("Rendimiento por cada acción (basado en tus datos):\n")
for index, row in df.iterrows():
    print(f"{row['Ticker']}: {row['Rendimiento']:.2%}")

print("\n" + "="*40)
print(f"Rendimiento total del portafolio: {rendimiento_portafolio:.2%}")
print(f"Varianza de rendimientos: {varianza:.6f}")
print(f"Desviación estándar de rendimientos: {desviacion:.6f}")


Rendimiento por cada acción (basado en tus datos):

GCARSOA1: 3.86%
GFINBUR: 1.53%
TELEVISA: -2.70%
PINFRA: 0.90%
HOTEL: 1.45%

Rendimiento total del portafolio: 1.34%
Varianza de rendimientos: 0.000559
Desviación estándar de rendimientos: 0.023648


In [9]:
import yfinance as yf
import numpy as np
import pandas as pd

# Definición de los tickers y los pesos del portafolio
tickers = ["HOTEL.MX", "PINFRA.MX", "TLEVISACPO.MX", "GFINBURO.MX", "GCARSOA1.MX"]
pesos = np.array([0.1, 0.2, 0.15, 0.3, 0.25])

# Definición de las fechas de inicio y fin
start_date = "2025-09-08"
end_date = "2025-09-26" # Se pone 26 para incluir el 25

# Descargar los precios de cierre ajustados
data = yf.download(tickers, start=start_date, end=end_date)['Adj Close']

# 1. Calcular los rendimientos diarios
rendimientos_diarios = data.pct_change().dropna()

# 2. Calcular la matriz de covarianza anualizada
# Se multiplica por 252, que es el número aproximado de días de operación en un año.
matriz_covarianza = rendimientos_diarios.cov() * 252

# 3. Calcular la varianza del portafolio
# Fórmula: wT * Σ * w (transpuesta de pesos * matriz de covarianza * pesos)
varianza_portafolio = np.dot(pesos.T, np.dot(matriz_covarianza, pesos))

# 4. Calcular la desviación estándar (volatilidad) del portafolio
desviacion_estandar_portafolio = np.sqrt(varianza_portafolio)

# --- Impresión de Resultados ---
print("Cálculo del Riesgo del Portafolio (Anualizado)\n")
print(f"Varianza del Portafolio: {varianza_portafolio:.4f}")
print(f"Desviación Estándar (Riesgo/Volatilidad): {desviacion_estandar_portafolio:.2%}")

C:\Users\Juan Marco\AppData\Local\Temp\ipykernel_21100\3518291429.py:14:FutureWarning: YF.download() has changed argument auto_adjust default to True
[*********************100%***********************]  5 of 5 completed



KeyError: 'Adj Close'